<a href="https://colab.research.google.com/github/medinadiegoeverardo/DS-Unit-2-Regression-Classification/blob/master/module2/medinadiego_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [2]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module2')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 19.30 MiB | 16.76 MiB/s, done.
Resolving deltas: 100% (71/71), done.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 92kB 5.9MB/s 
     |████████████████████████████████| 112kB 16.4MB/s 
     |████████████████████████████████| 133kB 13.2MB/s 
     |████████████████████████████████| 57.7MB 1.3MB/s 
     |████████████████████████████████| 235kB 51.8MB/s 
     |████████████████████████████████| 614kB 50.3MB/s 
     |████████████████████████████████| 235kB 33.2MB/s 
     |████████████████████████████████| 307kB 53.5MB/s 
     |████████████████████████████████| 215kB 52.6MB/s 
     |████████████████████████████████| 51kB 12.8MB/s 
     |█████████

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
df.dtypes

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

In [5]:
df.head(5)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
# exploring data: (to give an idea of which variable may affect y the most? (skewing data))
get_describes = [df[x].describe() for x in df.columns]
print(get_describes)

[count    48817.000000
mean         1.201794
std          0.470711
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         10.000000
Name: bathrooms, dtype: float64, count    48817.000000
mean         1.537149
std          1.106087
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          8.000000
Name: bedrooms, dtype: float64, count                   48817
unique                  48148
top       2016-05-17 14:29:08
freq                        3
Name: created, dtype: object, count        47392
unique       37853
top               
freq          1627
Name: description, dtype: object, count        48684
unique        8674
top       Broadway
freq           435
Name: display_address, dtype: object, count    48817.000000
mean        40.750760
std          0.038954
min         40.575700
25%         40.728300
50%         40.751700
75%         40.774000
max         40.989400
Name: latitude, dtype: float64, 

## Mean Baseline

In [19]:
# baseline for y
initital_baseline = df['price'].mean()
print('Predicted price ${0:.2f}.'.format(df['price'].mean()))

Predicted price $3579.59.


In [29]:
# train baseline for x effect on y
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# mean absolute error for Train Error
y_train_test = df['price'] # split after I come up with feature engineering
y_train_prediction = [initital_baseline] * len(y_train_test)
mae = mean_absolute_error(y_train_test, y_train_prediction)
print(f"Our training model's MA error is: ${mae:,.0f}")

Our training model's error is: $1,202


In [228]:
# mean absolute error for Test Error
y_test_test = df['price'] # split after I come up with feature engineering
y_test_prediction = [initital_baseline] * len(y_train_test)
mae_1 = mean_absolute_error(y_test_test, y_test_prediction)
print(f"Our testing model's MA error is: ${mae_1:,.0f}")

ValueError: ignored

In [34]:
# mean squared error for both Training and Test
squared_train_error = np.sqrt(mean_squared_error(y_train_test, y_train_prediction)) # split after I come up with feature engineering
squared_test_error = np.sqrt(mean_squared_error(y_test_test, y_test_prediction)) # split after I come up with feature engineering
print(f"Our Training model's squared error is: ${squared_train_error:,.0f}")
print(f"Our Testing model's squared error is: ${squared_test_error:,.0f}")

Our Training model's squared error is: $1,762
Our Testing model's squared error is: $1,762


In [36]:
# r^2 score for both Training and Test
r_training_score = r2_score(y_train_test, y_train_prediction)
r_testing_score = r2_score(y_test_test, y_test_prediction)
print('R-score Training error is: {0:.4f}.'.format(r_training_score))
print('R-score Testing error is: {0:.4f}.'.format(r_testing_score))

R-score Training error is: 0.0000.
R-score Testing error is: 0.0000.


## 1 feature (baseline)

In [39]:
from sklearn.linear_model import LinearRegression
model_1 = LinearRegression()

# features (matrix)
x_train_feature = df[['bedrooms']] # why does this need double brackets again?

# target (vector)
y_train_target = df['price']

model_1.fit(x_train_feature, y_train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
# coef and intercept
# What do these mean: Coefficient is the slope ('rate per unit'). Intercept is location on y axis
print('Coefficient is {0} \n Intercept is {1}'.format(model_1.coef_, model_1.intercept_))

Coefficient is [853.26640848] 
 Intercept is 2267.987688178934


In [0]:
#model_1.predict()

def gives_predictions(model, value):
  prediction_values = [[value] for value in len(value)]
  model_predictions = model.predict(prediction_values)
  return model_predictions

  # gives_predictions(model_1, [1, 2, 3])
  # (not working) will come back to this

In [63]:
# just for clarity / visualization
model_predictions = model_1.predict([[1], [2], [3], [4]])
print('1 bedroom prediction {},\n 2 bedroom prediction {},\n 3 bedroom prediction {}'.format(model_predictions[0], model_predictions[1], model_predictions[2]))

1 bedroom prediction 3121.254096662109,
 2 bedroom prediction 3974.520505145284,
 3 bedroom prediction 4827.786913628459


In [82]:
len(y_training_pred) # just making sure everything looks good. (prediction len has to be the same as y column (df['price']))

48817


In [87]:
# Train Mean absolute error
y_training_pred = model_1.predict(x_train_feature)
mae = mean_absolute_error(y_train_test, y_training_pred) # y_train_test is df['price'] remember
print('On average, we are off by ${0:.2f}'.format(mae))

On average, we are off by $975.65


## Feature engineering

In [0]:
# to date to datetime
condition = pd.to_datetime(df['created'])
df['created'] = condition

In [0]:
# encoding interest level
interest_level_map = {'low': 1, 'medium': 2, 'high':3}
df['interest_level'] = df['interest_level'].replace(interest_level_map) 

In [0]:
# encoding bedrooms
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, OneHotEncoder

one_hot = LabelBinarizer()
# to_encode = ['bathrooms', 'bedrooms']
encoded_bed = one_hot.fit_transform(df[['bedrooms']])
encoded_bedrooms = pd.DataFrame(encoded_bed)
#pd.concat([df, encoded_bedrooms], axis=1)

In [227]:
encoded_bedrooms.shape

(48817, 9)

In [225]:
df = pd.concat([df, encoded_bedrooms], axis=1)
df.shape

(49350, 52)

In [0]:
new_df = df.drop(columns=['description', 'display_address', 'street_address'])

In [226]:
new_df.head()
print(new_df.shape)

(49350, 40)


In [0]:
# split data into training and test
april_may_train = new_df[(new_df['created'] > '2016-04-01') & (new_df['created'] <= '2016-05-30')]
june_test = new_df[(new_df['created'] > '2016-06-01') & (new_df['created'] < '2016-06-30')]

## 2+ features

In [0]:
# new_df.isnull().sum()

In [0]:
new_df = new_df.dropna(axis=0)

In [234]:
new_df.shape

(48284, 40)

In [235]:
from sklearn.linear_model import LinearRegression
model_2 = LinearRegression()

y = ['price', 'created']
x_test_feature = [[x for x in new_df.columns if x not in y]]
y_test_target = df['price']

model_2.fit(x_test_feature, y_test_target)
# Input contains NaN, infinity or a value too large for dtype('float64').

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:532: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)


ValueError: ignored